<a href="https://colab.research.google.com/github/jedhouas/Mini-projet_Deep_Reinforcement_Learning/blob/master/DQN_project_MVA_jed_houas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [70]:
!pip install sk-video

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
  def __init__(self, epsilon=0.1, n_action=4):
      self.epsilon = epsilon
      self.n_action = n_action
  
  def set_epsilon(self,e):
      self.epsilon = e

  def act(self,s,train=True):
      """ This function should return the next action to do:
      an integer between 0 and 4 (not included) with a random exploration of epsilon"""
      if train:
          if np.random.rand() <= self.epsilon:
              a = np.random.randint(0, self.n_action, size=1)[0]
          else:
              a = self.learned_act(s)
      else: # in some cases, this can improve the performance.. remove it if poor performances
          a = self.learned_act(s)

      return a

  def learned_act(self,s):
      """ Act via the policy of the agent, from a given state s
      it proposes an action a"""
      pass

  def reinforce(self, s, n_s, a, r, game_over_):
      """ This function is the core of the learning algorithm. 
      It takes as an input the current state s_, the next state n_s_
      the action a_ used to move from s_ to n_s_ and the reward r_.
      
      Its goal is to learn a policy.
      """
      pass

  def save(self):
      """ This function returns basic stats if applicable: the
      loss and/or the model"""
      pass

  def load(self):
      """ This function allows to restore a model"""
      pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The **act** function returns what action the agent will perform knowing the current state. As defined in the code above, this action is following an $\epsilon$-greedy policy: with a probability $\epsilon$ we explore (choose randomly an action) and with $1 - \epsilon$ we act based on what we have learned on all the previous cumulated knowledge.
$$
\pi(s) = \begin{cases}
\max_a Q(s,a) & \text{w.p.} 1- \epsilon\\
\text{random action} & \text{w.p.} \epsilon
\end{cases}
$$
**Epsilon** is essential to add **exploration** to the behavior of the agent: if he acts greedily no exploration will be performed if it causes a lower reward and so we won't be able to explore the environnement which is eessential to learn an optimal policy.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=40 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

 

*   The **position** array contains an array representing where the rat can go. It has -1 in the boundaries where we can't go, 0 where we can go and 1 in the position where the rat is actually at.
*   The **board** array represents the reward associated to each cell of the grid. If the rat comes to the position $(x,y)$ it will get the reward $board[x,y]$.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
  def __init__(self):
      super(RandomAgent, self).__init__()
      
  def learned_act(self, s):
      return np.random.randint(0,self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
 
    for e in range(epochs):
        
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        
        while not game_over:
            
            action = agent.act(state,train=False) #this is the main point, during the test phase we don't have to use the epsilon greedy approach
            
            prev_state= state
            state,reward,game_over = env.act(action)
            if reward >0:
                win = win +reward 
            if reward < 0:
                lose = lose-reward
                
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [79]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 6.5/15.0. Average score (-8.5)
Win/lose count 5.5/15.0. Average score (-9.0)
Win/lose count 16.0/10.0. Average score (-4.0)
Win/lose count 9.5/6.0. Average score (-2.125)
Win/lose count 10.0/15.0. Average score (-2.7)
Win/lose count 12.0/17.0. Average score (-3.0833333333333335)
Win/lose count 8.0/11.0. Average score (-3.0714285714285716)
Win/lose count 13.0/17.0. Average score (-3.1875)
Win/lose count 8.5/17.0. Average score (-3.7777777777777777)
Win/lose count 8.5/16.0. Average score (-4.15)
Final score: -4.15


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




#### First for the Bellman equation:

\begin{align*}
Q^\pi(s,a)&=E_{p^{\pi}}[\sum_{0\leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a,a_t=\pi(s_t)].\\
&=E_{p^{\pi}}[r(s,a)+\gamma \sum_{1\leq t}\gamma^{t-1}r(s_{t},a_{t})]|s_{0}=s,a_{0}=a,a_t=\pi(s_t)] \\
&=E_{p^{\pi}}[r(s,a)+\gamma \sum_{0\leq t}\gamma^{t}r(s_{t+1},a_{t+1})]|s_{0}=s,a_{0}=a,a_t=\pi(s_t)] \\
&=E_{p^{\pi}}[r(s,a)+\gamma \sum_{s',a'}\left( p(s1=s',a1=a') E[ \sum_{0\leq t}\gamma^{t}r(s_{t+1},a_{t+1})]|s_{1}=s',a_{1}=a',a_t=\pi(s_t)] \right)|s_{0}=s,a_{0}=a,a_t=\pi(s_t)] \\
&=r(s,a)+\gamma \sum_{s',a'}\left( p(s_1=s',a_1=a'|s_{0}=s,a_{0}=a) E[ \sum_{0\leq t}\gamma^{t}r(s_{t+1},a_{t+1})]|s_{1}=s',a_{1}=a',a_t=\pi(s_t)] \right) \\
&=r(s,a)+\gamma \sum_{s',a'} p(s_1=s',a_1=a'|s_{0}=s,a_{0}=a) Q^\pi(s',a') \\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align*}

#### As for the Bellman optimal equation:
\begin{align*}
Q^{*}(s,a) &= \max_{\pi}E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\
&= \max_{\pi}r(s,a) + \gamma \sum_{(s',a')}(p(s_{1}=s',a_{1}=a' |s_0 = s, a_0 = a )Q^{\pi}(s',a')\\
&= r(s,a) + \gamma \max_{\pi} \sum_{(s',a')}(p(s_{1}=s',a_{1}=a' |s_0 = s, a_0 = a )Q^{\pi}(s',a') \\
&= r(s,a) + \gamma \sum_{s'}(p(s_{1}=s'|s_0 = s, a_0 = a )\max_{(a',\pi')} Q^{\pi'}(s',a') \\
&= r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'} Q^{*}(s',a') \\ 
&=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{align*}

The proposed objective function is plausible because it enforces this last equation we wrote (the Bellman optimal equation): in fact $E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$ is the distance between the two quantities of this equation.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>self.max_memory:
            del self.memory[0]

    def random_access(self):
        return self.memory[np.random.randint(0, len(self.memory), size=1)[0]]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        # Improve the prediction where we do have new information
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
      
            if game_over_:
                input_states[i] = s_
                target_q[i,a_] = r_
            else:
                input_states[i] = s_
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0])
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class DQN_FC_1L(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC_1L, self).__init__( *args,**kwargs)
        
        # NN Model: One hidden layer
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(30,activation ='relu'))
        model.add(Dense(4, activation = None))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

class DQN_FC_5L(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC_5L, self).__init__( *args,**kwargs)
        
        # NN Model: Five hidden layers
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(32,activation ='relu'))
        model.add(BatchNormalization())
        model.add(Dense(24,activation ='relu'))
        model.add(BatchNormalization())
        model.add(Dense(16,activation ='relu'))
        model.add(BatchNormalization())
        model.add(Dense(8,activation ='relu'))
        model.add(BatchNormalization())
        model.add(Dense(4, activation = None))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [0]:
# Testing the One layer model with lr of 0.1
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC_1L(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train_best')
HTML(display_videos('fc_train_best10.mp4'))

Epoch 000/040 | Loss 0.0117 | Win/lose count 4.0/4.0 (0.0)
Epoch 001/040 | Loss 0.0131 | Win/lose count 3.0/3.0 (0.0)
Epoch 002/040 | Loss 0.0082 | Win/lose count 3.5/3.0 (0.5)
Epoch 003/040 | Loss 0.0545 | Win/lose count 3.5/4.0 (-0.5)
Epoch 004/040 | Loss 0.0059 | Win/lose count 5.5/1.0 (4.5)
Epoch 005/040 | Loss 0.0080 | Win/lose count 2.5/1.0 (1.5)
Epoch 006/040 | Loss 0.0050 | Win/lose count 3.5/1.0 (2.5)
Epoch 007/040 | Loss 0.0014 | Win/lose count 3.5/2.0 (1.5)
Epoch 008/040 | Loss 0.0050 | Win/lose count 1.0/1.0 (0.0)
Epoch 009/040 | Loss 0.0031 | Win/lose count 5.0/4.0 (1.0)
Epoch 010/040 | Loss 0.0021 | Win/lose count 3.0/1.0 (2.0)
Epoch 011/040 | Loss 0.0063 | Win/lose count 5.0/2.0 (3.0)
Epoch 012/040 | Loss 0.0044 | Win/lose count 3.5/6.0 (-2.5)
Epoch 013/040 | Loss 0.0030 | Win/lose count 1.5/2.0 (-0.5)
Epoch 014/040 | Loss 0.0089 | Win/lose count 1.5/6.0 (-4.5)
Epoch 015/040 | Loss 0.0159 | Win/lose count 5.0/5.0 (0.0)
Epoch 016/040 | Loss 0.0098 | Win/lose count 6.0/4.0

In [0]:
# Testing the One layer model with lr of 0.01
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC_1L(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/040 | Loss 0.0101 | Win/lose count 1.5/2.0 (-0.5)
Epoch 001/040 | Loss 0.0086 | Win/lose count 2.5/6.0 (-3.5)
Epoch 002/040 | Loss 0.0306 | Win/lose count 2.5/4.0 (-1.5)
Epoch 003/040 | Loss 0.0149 | Win/lose count 3.0/5.0 (-2.0)
Epoch 004/040 | Loss 0.0228 | Win/lose count 4.5/10.0 (-5.5)
Epoch 005/040 | Loss 0.0147 | Win/lose count 3.5/4.0 (-0.5)
Epoch 006/040 | Loss 0.0097 | Win/lose count 4.5/7.0 (-2.5)
Epoch 007/040 | Loss 0.0122 | Win/lose count 2.0/4.0 (-2.0)
Epoch 008/040 | Loss 0.0249 | Win/lose count 4.0/5.0 (-1.0)
Epoch 009/040 | Loss 0.0210 | Win/lose count 5.5/5.0 (0.5)
Epoch 010/040 | Loss 0.0209 | Win/lose count 5.0/1.0 (4.0)
Epoch 011/040 | Loss 0.0271 | Win/lose count 2.5/0 (2.5)
Epoch 012/040 | Loss 0.0144 | Win/lose count 2.5/4.0 (-1.5)
Epoch 013/040 | Loss 0.0228 | Win/lose count 2.0/3.0 (-1.0)
Epoch 014/040 | Loss 0.0263 | Win/lose count 7.0/10.0 (-3.0)
Epoch 015/040 | Loss 0.0303 | Win/lose count 4.0/1.0 (3.0)
Epoch 016/040 | Loss 0.0083 | Win/lose count

In [0]:
# Testing the Five layer model (with memory size of 2000 and lr of 0.1)
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC_5L(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))


Epoch 000/040 | Loss 0.1192 | Win/lose count 7.5/15.0 (-7.5)
Epoch 001/040 | Loss 0.0167 | Win/lose count 6.5/9.0 (-2.5)
Epoch 002/040 | Loss 0.0424 | Win/lose count 6.0/10.0 (-4.0)
Epoch 003/040 | Loss 0.0200 | Win/lose count 4.5/12.0 (-7.5)
Epoch 004/040 | Loss 0.0318 | Win/lose count 3.0/15.0 (-12.0)
Epoch 005/040 | Loss 0.0180 | Win/lose count 9.0/7.0 (2.0)
Epoch 006/040 | Loss 0.0201 | Win/lose count 6.5/8.0 (-1.5)
Epoch 007/040 | Loss 0.0235 | Win/lose count 3.0/5.0 (-2.0)
Epoch 008/040 | Loss 0.0272 | Win/lose count 7.0/16.0 (-9.0)
Epoch 009/040 | Loss 0.0060 | Win/lose count 6.0/6.0 (0.0)
Epoch 010/040 | Loss 0.0074 | Win/lose count 7.0/5.0 (2.0)
Epoch 011/040 | Loss 0.0288 | Win/lose count 7.5/5.0 (2.5)
Epoch 012/040 | Loss 0.0144 | Win/lose count 9.0/4.0 (5.0)
Epoch 013/040 | Loss 0.0152 | Win/lose count 7.5/3.0 (4.5)
Epoch 014/040 | Loss 0.0170 | Win/lose count 7.0/10.0 (-3.0)
Epoch 015/040 | Loss 0.0088 | Win/lose count 6.0/3.0 (3.0)
Epoch 016/040 | Loss 0.0101 | Win/lose 

After testing the two model with different parameters, We choose the 1 layer model with relu activation, lr of 0.1, memory size of 2000, batch size of 32 and $\epsilon = 0.1$

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(30,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

After testing many CNN architecture and different parameters, we choose the model with 2 hidden Convolution Layers (50,(2,2)) and (30,(2,2)), relu activations, lr of 0.1 and memory size of 2000, batch size of 32 and $\epsilon = 0.1$

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train_best')
HTML(display_videos('cnn_train_best10.mp4'))

Epoch 000/040 | Loss 0.0012 | Win/lose count 4.0/6.0 (-2.0)
Epoch 001/040 | Loss 0.0014 | Win/lose count 2.5/5.0 (-2.5)
Epoch 002/040 | Loss 0.0037 | Win/lose count 0.5/2.0 (-1.5)
Epoch 003/040 | Loss 0.0034 | Win/lose count 1.0/2.0 (-1.0)
Epoch 004/040 | Loss 0.0094 | Win/lose count 5.5/5.0 (0.5)
Epoch 005/040 | Loss 0.0085 | Win/lose count 4.5/6.0 (-1.5)
Epoch 006/040 | Loss 0.0022 | Win/lose count 4.5/2.0 (2.5)
Epoch 007/040 | Loss 0.0216 | Win/lose count 2.0/7.0 (-5.0)
Epoch 008/040 | Loss 0.0055 | Win/lose count 3.0/1.0 (2.0)
Epoch 009/040 | Loss 0.0063 | Win/lose count 6.0/0 (6.0)
Epoch 010/040 | Loss 0.0019 | Win/lose count 4.0/0 (4.0)
Epoch 011/040 | Loss 0.0018 | Win/lose count 7.5/5.0 (2.5)
Epoch 012/040 | Loss 0.0014 | Win/lose count 5.0/1.0 (4.0)
Epoch 013/040 | Loss 0.0048 | Win/lose count 7.5/1.0 (6.5)
Epoch 014/040 | Loss 0.0564 | Win/lose count 5.5/3.0 (2.5)
Epoch 015/040 | Loss 0.1081 | Win/lose count 4.0/2.0 (2.0)
Epoch 016/040 | Loss 0.0028 | Win/lose count 5.0/4.0 (

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
# Testing with temperature 0.3
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_train_bestmodel.h5',name_model='cnn_train_bestmodel.json')

agent_fc = DQN_FC_1L(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_train_bestmodel.h5',name_model='fc_train_bestmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 6.0/1.0. Average score (5.0)
Win/lose count 7.0/0. Average score (6.0)
Win/lose count 11.0/0. Average score (7.666666666666667)
Win/lose count 4.5/0. Average score (6.875)
Win/lose count 1.5/0. Average score (5.8)
Win/lose count 21.0/0. Average score (8.333333333333334)
Win/lose count 5.0/0. Average score (7.857142857142857)
Win/lose count 5.5/0. Average score (7.5625)
Win/lose count 6.0/0. Average score (7.388888888888889)
Win/lose count 7.5/0. Average score (7.4)
Final score: 7.4
Test of the FC
Win/lose count 5.0/0. Average score (5.0)
Win/lose count 3.0/0. Average score (4.0)
Win/lose count 2.5/0. Average score (3.5)
Win/lose count 1.0/0. Average score (2.875)
Win/lose count 4.5/0. Average score (3.2)
Win/lose count 0.5/0. Average score (2.75)
Win/lose count 9.0/0. Average score (3.642857142857143)
Win/lose count 3.0/0. Average score (3.5625)
Win/lose count 5.5/0. Average score (3.7777777777777777)
Win/lose count 2.0/0. Average score (3.6)
Final score:

In [0]:
HTML(display_videos('cnn_test0.mp4'))

In [0]:
HTML(display_videos('fc_test0.mp4'))

In [0]:
# Testing with temperature 0.5
env = Environment(grid_size=size, max_time=T,temperature=0.6)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_train_bestmodel.h5',name_model='cnn_train_bestmodel.json')

agent_fc = DQN_FC_1L(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_train_bestmodel.h5',name_model='fc_train_bestmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 3.5/0. Average score (3.5)
Win/lose count 17.0/0. Average score (10.25)
Win/lose count 13.5/0. Average score (11.333333333333334)
Win/lose count 12.0/1.0. Average score (11.25)
Win/lose count 19.0/0. Average score (12.8)
Win/lose count 16.5/0. Average score (13.416666666666666)
Win/lose count 27.5/1.0. Average score (15.285714285714286)
Win/lose count 16.5/0. Average score (15.4375)
Win/lose count 17.5/0. Average score (15.666666666666666)
Win/lose count 9.0/0. Average score (15.0)
Final score: 15.0
Test of the FC
Win/lose count 2.5/0. Average score (2.5)
Win/lose count 4.0/0. Average score (3.25)
Win/lose count 5.5/0. Average score (4.0)
Win/lose count 11.5/1.0. Average score (5.625)
Win/lose count 5.5/0. Average score (5.6)
Win/lose count 3.0/0. Average score (5.166666666666667)
Win/lose count 8.0/0. Average score (5.571428571428571)
Win/lose count 2.0/1.0. Average score (5.0)
Win/lose count 0.5/0. Average score (4.5)
Win/lose count 19.5/1.0. Average sc

In [0]:
# Testing with temperature 0.9
env = Environment(grid_size=size, max_time=T,temperature=0.9)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_train_bestmodel.h5',name_model='cnn_train_bestmodel.json')

agent_fc = DQN_FC_1L(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_train_bestmodel.h5',name_model='fc_train_bestmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 29.5/0. Average score (29.5)
Win/lose count 27.5/0. Average score (28.5)
Win/lose count 12.0/0. Average score (23.0)
Win/lose count 27.5/0. Average score (24.125)
Win/lose count 11.5/0. Average score (21.6)
Win/lose count 48.0/0. Average score (26.0)
Win/lose count 48.0/0. Average score (29.142857142857142)
Win/lose count 19.0/1.0. Average score (27.75)
Win/lose count 53.5/0. Average score (30.61111111111111)
Win/lose count 20.5/0. Average score (29.6)
Final score: 29.6
Test of the FC
Win/lose count 19.5/0. Average score (19.5)
Win/lose count 6.0/1.0. Average score (12.25)
Win/lose count 25.5/1.0. Average score (16.333333333333332)
Win/lose count 14.5/1.0. Average score (15.625)
Win/lose count 19.0/0. Average score (16.3)
Win/lose count 19.5/0. Average score (16.833333333333332)
Win/lose count 27.5/0. Average score (18.357142857142858)
Win/lose count 11.0/0. Average score (17.4375)
Win/lose count 15.0/0. Average score (17.166666666666668)
Win/lose count 1

The performance using CNN is better than the one obtained using  fully connected networks and it is better at avoiding the poisonnous states.  **This gap is increasing with the temperature** as we can see.

The main issue that we can observe is that the algorithms tends to **not explore the map** which means that sometimes it goes to a position A then goes to a position B and then go back to position A (most of the time they get stucked in a left-right or up-down loop). This method will not decrease the score but will not help to improve it.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,epsilon_decay=1,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1, epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Decrease the exploration rate
        agent.set_epsilon(agent.epsilon * epsilon_decay)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, round(win, 2), round(lose, 2), round(win-lose, 2)))
        agent.save(name_weights=prefix+'model.h5', name_model=prefix+'model.json')

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self, e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self, t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        # Exploration malus
        if train:
            reward = -self.malus_position[self.x, self.y] # Punish the agent if the state was visited before
        self.malus_position[self.x, self.y] = 0.1 # Mark the current state as visited
        # Normal bonus
        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        # The state now includes wether the surrounding states were visited before
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
    
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
### use those samples of code:
##In train explore:
#state, reward, game_over = env.act(action, train=True)
#
### In Environment exploring:
## You will have to change n_state to 3 because you will use one more layer!
#reward = 0
#if train:
#    reward = -self.malus_position[self.x, self.y]
#self.malus_position[self.x, self.y] = 0.1
#
#reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
#state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                self.board.reshape(self.grid_size, self.grid_size,1),
#                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [85]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 001/040 | Loss 0.0023 | Win/lose count 6.0/21.3 (-15.3)
Epoch 002/040 | Loss 0.0015 | Win/lose count 3.7/20.4 (-16.7)
Epoch 003/040 | Loss 0.0079 | Win/lose count 3.5/20.3 (-16.8)
Epoch 004/040 | Loss 0.0037 | Win/lose count 3.8/21.7 (-17.9)
Epoch 005/040 | Loss 0.0010 | Win/lose count 7.6/22.7 (-15.1)
Epoch 006/040 | Loss 0.0069 | Win/lose count 9.6/20.6 (-11.0)
Epoch 007/040 | Loss 0.0351 | Win/lose count 2.9/22.2 (-19.3)
Epoch 008/040 | Loss 0.0038 | Win/lose count 5.8/22.6 (-16.8)
Epoch 009/040 | Loss 0.0233 | Win/lose count 6.1/19.5 (-13.4)
Epoch 010/040 | Loss 0.0039 | Win/lose count 3.3/22.1 (-18.8)
Epoch 011/040 | Loss 0.0304 | Win/lose count 2.8/20.4 (-17.6)
Epoch 012/040 | Loss 0.0156 | Win/lose count 6.8/18.1 (-11.3)
Epoch 013/040 | Loss 0.0041 | Win/lose count 8.0/21.1 (-13.1)
Epoch 014/040 | Loss 0.0036 | Win/lose count 8.1/19.9 (-11.8)
Epoch 015/040 | Loss 0.0089 | Win/lose count 8.9/19.7 (-10.8)
Epoch 016/040 | Loss 0.0045 | Win/lose count 12.5/18.8 (-6.3)
Epoch 01

In [87]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Win/lose count 9.0/1.0. Average score (8.0)
Win/lose count 13.5/0. Average score (10.75)
Win/lose count 21.0/0. Average score (14.166666666666666)
Win/lose count 12.0/1.0. Average score (13.375)
Win/lose count 18.5/1.0. Average score (14.2)
Win/lose count 12.0/1.0. Average score (13.666666666666666)
Win/lose count 14.0/1.0. Average score (13.571428571428571)
Win/lose count 7.0/0. Average score (12.75)
Win/lose count 16.5/0. Average score (13.166666666666666)
Win/lose count 8.5/0. Average score (12.7)
Final score: 12.7


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***